In [ ]:
! pip install selenium
! pip install pandas

In [2]:
bus_routes =[]
bus_details =[]
# empty list for storing the collected routes

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://www.redbus.in/')

States = ['APSRTC',
          'KSRTC (Kerala)',
          'TSRTC',
          'Kadamba Transport Corporation Limited (KTCL)',
          'RSRTC',
          'South Bengal State Transport Corporation (SBSTC)',
          'HRTC',
          'Assam State Transport Corporation (ASTC)',
          'UPSRTC',
          'WBTC (CTC)',
          'PEPSU (Punjab)']
# select the states to scrap the routes

def collect_bus_routes_links(current_state):
            
    bus_route = driver.find_elements(By.XPATH,"//a[@class ='route']")
    route_links = driver.find_elements(By.XPATH,"//div[@class = 'route_details']")

    # if len(bus_route) != len(route_links):
    #     print("Error: Mismatch between number of routes and route links.")
    #     return
    
    for i in range(len(bus_route)):
        bus_RT = bus_route[i].text

        links = route_links[i].find_elements(By.TAG_NAME,"a")
        for link in links:
            href = link.get_attribute("href")
            
            bus_rts = {
                "State": current_state,
                "Routes": bus_RT,
                "Route_links": href
            }
            bus_routes.append(bus_rts)

# wrote this function to reuse code and
# collecting the bus routes data from the page and appending only the text to the empty list

def page_navigation():
    page_number = 2
    # created and assigned the variable to loop the page number

    while True:

        try:
            path = f"//div[@class='DC_117_pageTabs ' and contains(text(), '{page_number}')]"
            # path for the pages

            driver.find_element(By.XPATH,path).click()
            # navigating to the secone page

            collect_bus_routes_links(state)
            # function called and pages data collected

            page_number = page_number + 1
            # increment to navigate the second page
            # loop continue until the end page
            
        except Exception:
            break
        # if there is no other pages in routes this except will break the condition


def scrolling_the_page():

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Scroll down to the bottom
        
        time.sleep(0.5)
        # Wait for new elements to load
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        # Calculate new scroll height and compare with the last scroll height
        
        if new_height == last_height:  # Break if no more buses are loaded
            break
        last_height = new_height



for state in States:

    driver.get("https://www.redbus.in/online-booking/rtc-directory")
    # pass the link where we need to scrap the data
    # time.sleep(1)
    # wait 1 sec to load the web page

    driver.execute_script('window.scrollBy(0, 210)')
    time.sleep(1)

    path = f"{state}"
    routes = driver.find_element(By.LINK_TEXT, path).click()
    time.sleep(0.5)
    # finding the state and click to collect the data
    
    collect_bus_routes_links(state)
    # collecting the data in single page
    
    driver.execute_script('window.scrollBy(0, 1500)')
    time.sleep(1)

    page_navigation()


def bus_details_scraping(current_route,current_state,current_link):
    time.sleep(1)
    all_buses = driver.find_element(By.XPATH, "//div[@id='result-section']")
    li_elements = all_buses.find_elements(By.CSS_SELECTOR, "li.row-sec")
    # After scrolling, find all <li> elements within the <ul>

    for li in li_elements:
        bus_name = li.find_element(By.CSS_SELECTOR, ".travels").text # bus name
        bus_types = li.find_element(By.CSS_SELECTOR, ".bus-type").text # bus type
        dp_time = li.find_element(By.CSS_SELECTOR, ".dp-time").text # depart time
        dur = li.find_element(By.CSS_SELECTOR, ".dur").text # duration
        re_time = li.find_element(By.CSS_SELECTOR, ".bp-time").text # reach time 
        seat = li.find_element(By.CSS_SELECTOR, ".seat-left").text # seat available
        ratings = li.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']//span") # star rating

        if ratings:
            for rating in ratings:
                star_rating = rating.text  # Get the star rating
        else:
            star_rating = 0.0
            
        fare = li.find_element(By.CSS_SELECTOR, ".fare span.f-19").text # price

        
        bus_items = ({
            "States": current_state,
            "Route_link": current_link,
            "Routes_Name": current_route,
            "Bus_Name" : bus_name,
            "Bus_Type": bus_types,
            "Departing_Time": dp_time,
            "Duration": dur,
            "Reaching_Time": re_time,
            "Price":fare,
            "Seat_Availability": seat,
            "Star_Rating": star_rating,
        })
        bus_details.append(bus_items)

driver.quit()

In [ ]:
len(bus_routes)
# to check how many routes are there

In [15]:
# all states bus details scraping

driver = webdriver.Chrome()
driver.maximize_window()

for item in bus_routes:
    link = item['Route_links']
    route = item['Routes']
    states =item['State']
    # looping the links for route to collect the bus details

    try:
        try:
            driver.get(link)
            time.sleep(5)

            try:
                driver.find_element(By.XPATH, "(//div[@class = 'button' and contains(text(), 'View Buses')])[2]").click()
                time.sleep(1.5)
                driver.find_element(By.XPATH, "//div[@class = 'button' and contains(text(), 'View Buses')]").click()

            except:
                driver.find_element(By.XPATH, "//div[@class = 'button' and contains(text(), 'View Buses')]").click()
                time.sleep(1)
                
            scrolling_the_page()
            bus_details_scraping(route,states,link)

        except Exception:
            driver.get(link)
            time.sleep(4)
            scrolling_the_page()
            bus_details_scraping(route,states,link)
            
    except Exception:
        pass

driver.quit()


In [16]:
len(bus_details)
# to check how many buses details are scraped

11800

In [17]:
import pandas as pd

redbus_data = pd.DataFrame(bus_details)
# changing the list of dict to dataframe

len(redbus_data)

redbus_data.to_csv('Redbus_data_3.csv')
# convert the dataframe to csv for local storage


In [20]:
redbus_data
# view dataframe

,States,Route_link,Routes_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Price,Seat_Availability,Star_Rating
0,APSRTC,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,TSRTC - 40331,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",13:40,04h 50m,18:30,396,9 Seats available,3.3
1,APSRTC,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,TSRTC - 1600,RAJDHANI (A.C. Semi Sleeper),14:00,05h 40m,19:40,480,31 Seats available,2.6
2,APSRTC,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,TSRTC - 1432,RAJDHANI (A.C. Semi Sleeper),14:20,04h 55m,19:15,480,34 Seats available,2.1
3,APSRTC,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,TSRTC - 1402,RAJDHANI (A.C. Semi Sleeper),14:40,04h 50m,19:30,480,32 Seats available,2.1
4,APSRTC,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,TSRTC - 30162,GARUDA PLUS (VOLVO / BENZ A.C Multi Axle),15:00,06h 00m,21:00,533,34 Seats available,3.7
...,...,...,...,...,...,...,...,...,...,...,...
11795,PEPSU (Punjab),https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Bathinda,Orbit Aviation Pvt. Ltd.,Mercedes Benz A/C Sleeper (2+1),21:15,04h 15m,01:30,1050,42 Seats available,0.0
11796,PEPSU (Punjab),https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Bathinda,Rajpreet Travels and cargo,VE A/C Sleeper (2+1),21:20,04h 40m,02:00,905,36 Seats available,0.0
11797,PEPSU (Punjab),https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Faridkot,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),16:20,04h 25m,20:45,600,36 Seats available,4.3
11798,PEPSU (Punjab),https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Patiala,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),16:55,01h 35m,18:30,200,35 Seats available,3.6


In [23]:
import mysql.connector as db

db_connection = db.connect(
    host = "localhost",
    user = "NewUser",
    password = "root",
    database = "redbus_data"
)
# connect the MySQL localhost databases

In [25]:
curr = db_connection.cursor()
# make connection

In [27]:
create_table_query = """
CREATE TABLE redbus_data.bus_routes(
             Id INT AUTO_INCREMENT primary key,
             States varchar(100),
             Route_link varchar(100),
             Routes_Name varchar(100),
             Bus_Name varchar(100),
             Bus_Type varchar(100),
             Departing_Time time,
             Duration varchar(100),
             Reaching_Time time,
             Price decimal,
             Seat_Availability varchar(100),
             Star_Rating float
             );
"""
curr.execute(create_table_query)

# create empty table and add columns that contain previous DataFrame

In [28]:
insert_query = """
INSERT INTO redbus_data.bus_routes(States,Route_link,Routes_Name,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Price,Seat_Availability,Star_Rating)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

for _,row in redbus_data.iterrows():
    curr.execute(insert_query, tuple(row))
# loop through the rows in dataframe and add to the created table into the database
db_connection.commit()
# make commit to update

In [29]:
curr.close()
db_connection.close()
# finally close the all databases connections